목차  

[1.소개](#소개)  
[2.패키지 로딩](#패키지-로딩)  
[3.데이터 로딩](#데이터-로딩)  

# 소개

이 노트북은 PorteSeguro 대회에 대한 데이터의 insight를 얻는 것이 목적이다.  
그 외에 데이터 모델링에 대한 몇가지 팁과 재주를 얻을 수 있다. 이 노트북은 아래와 같은 세션으로 구성되어 있다.  

1. 데이터 시각화(Defining the metadata)
2. metadata 정의(Defining the metadata)
3. 기술적인 통계(Descriptive statistics)
4. ibalanced class들을 다루기(Handling imbalanced classes)
5. 데이터 질 확인(Data quality checks)
6. 탐색적 데이터 시각화(Exploratory data visualization)
7. 기계 학습(Feature engineering)
8. 특징 선택(Feature selection)
9. 데이터 시각화(Feature scaling)


# 패키지 로딩

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer #sklearn.preprocessing.Imputer 클래스가 삭제되어 sklearn.impute.SimpleImputer로 대체함
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

# 데이터 로딩

In [5]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

# 데이터 인사이트(1)